In [254]:
import numpy as np
import pandas as pd
import os

In [255]:
Balance_Sheet = ["Reserves and Surplus",
                 "Total Assets",
                 "Total Capital And Liabilities",
                 "Total Current Assets",
                 "Total Current Liabilities",
                 "Total Non-Current Assets",
                 "Total Reserves and Surplus",
                 "Total Share Capital",
                 "Total Shareholders Funds",
                 "Fixed Assets",
                 "Other Current Liabilities",
                 "Tangible Assets"]

Cash_Flow = ["Cash And Cash Equivalents Begin of Year",
             "Cash And Cash Equivalents End Of Year",
             "Net Cash Used In Investing Activities",
             "Net CashFlow From Operating Activities",
             "Net Profit/Loss Before Extraordinary Items And Tax",
             "Net Inc/Dec In Cash And Cash Equivalents",
             "Net Cash Used From Financing Activities",
             "Foreign Exchange Gains / Losses"]

Profit_Loss = ["Basic EPS (Rs.)",
               "Diluted EPS (Rs.)",
               "Employee Benefit Expenses",
               "Other Expenses",
               "Profit/Loss After Tax And Before ExtraOrdinary Items",
               "Profit/Loss Before Exceptional, ExtraOrdinary Items And Tax",
               "Profit/Loss Before Tax",
               "Profit/Loss For The Period","Profit/Loss From Continuing Operations",
               "Revenue From Operations [Gross]",
               "Revenue From Operations [Net]",
               "Total Expenses","Total Operating Revenues",
               "Total Revenue",
               "Other Income",
               "Depreciation And Amortisation Expenses",
               "Total Tax Expenses",
               "Deferred Tax",
               "Finance Costs",
               "Current Tax",
               "Operating And Direct Expenses"]

Ratio = ["Net Profit Margin (%)",
         "Net Profit/Share (Rs.)",
         "PBDIT Margin (%)",
         "PBDIT/Share (Rs.)",
         "PBIT Margin (%)",
         "PBIT/Share (Rs.)",
         "PBT Margin (%)",
         "PBT/Share (Rs.)",
         "Return on Assets (%)",
         "Return on Networth / Equity (%)",
         "Revenue from Operations/Share (Rs.)",
         "EV/EBITDA (X)","Enterprise Value (Cr.)",
         "Dividend Payout Ratio (CP) (%)",
         "Dividend Payout Ratio (NP) (%)",
         "Earnings Retention Ratio (%)",
         "Total Debt/Equity (X)"]

Year = ["Equity Share Capital",
        "Net Profit/(Loss) For the Period",
        "Other Expenses",
        "P/L After Tax from Ordinary Activities",
        "P/L Before Exceptional Items & Tax",
        "P/L Before Int., Excpt. Items & Tax",
        "P/L Before Other Inc. , Int., Excpt. Items & Tax",
        "P/L Before Tax","Employees Cost",
        "Net Sales/Income from operations",
        "Total Income From Operations",
        "Basic EPS","Basic EPS.",
        "Diluted EPS","Diluted EPS.",
        "depreciat",
        "Other Income",
        "Interest","Tax"]

Quarterly = ["Equity Share Capital",
        "Net Profit/(Loss) For the Period",
        "Other Expenses",
        "P/L After Tax from Ordinary Activities",
        "P/L Before Exceptional Items & Tax",
        "P/L Before Int., Excpt. Items & Tax",
        "P/L Before Other Inc. , Int., Excpt. Items & Tax",
        "P/L Before Tax","Employees Cost",
        "Net Sales/Income from operations",
        "Total Income From Operations",
        "Basic EPS","Basic EPS.",
        "Diluted EPS","Diluted EPS.",
        "depreciat",
        "Other Income",
        "Interest","Tax"]

In [256]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\3i Infotech Ltd\Excel\Cash-flow_combined.xlsx"
output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataCF.xlsx"

In [257]:
df = pd.read_excel(input_file)
df.head(3)

,Cash Flow of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,...,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths
1,Net Profit/Loss Before Extraordinary Items And...,161.40,104.14,51.05,41.00,19.87,-417.92,-379.45,-165.25,108.15,...,250.85,-86.71,91.09,-448.00,-343.56,-818.63,52.25,8.43,324.56,127.96
2,Net CashFlow From Operating Activities,356.00,199.67,111.26,5.16,30.62,-72.54,36.45,-81.36,233.73,...,86.52,82.20,113.86,35.39,73.99,14.63,108.00,-57.44,388.69,103.84


In [258]:
df.iloc[:, 0]

0                                                  NaN
1    Net Profit/Loss Before Extraordinary Items And...
2               Net CashFlow From Operating Activities
3                Net Cash Used In Investing Activities
4              Net Cash Used From Financing Activities
5                      Foreign Exchange Gains / Losses
6    Adjustments On Amalgamation Merger Demerger Ot...
7             Net Inc/Dec In Cash And Cash Equivalents
8              Cash And Cash Equivalents Begin of Year
9                Cash And Cash Equivalents End Of Year
Name: Cash Flow of 3i Infotech(in Rs. Cr.), dtype: object

In [259]:

# Extract df_parameters column
df_parameters = df.iloc[:, 0].astype(str)  # Convert to string to avoid NaN issues
df_parameters = list(df_parameters)

In [260]:
print(df_parameters)
print(len(df_parameters))

['nan', 'Net Profit/Loss Before Extraordinary Items And Tax', 'Net CashFlow From Operating Activities', 'Net Cash Used In Investing Activities', 'Net Cash Used From Financing Activities', 'Foreign Exchange Gains / Losses', 'Adjustments On Amalgamation Merger Demerger Others', 'Net Inc/Dec In Cash And Cash Equivalents', 'Cash And Cash Equivalents Begin of Year', 'Cash And Cash Equivalents End Of Year']
10


In [261]:
print(Balance_Sheet)
print(len(Balance_Sheet))

['Reserves and Surplus', 'Total Assets', 'Total Capital And Liabilities', 'Total Current Assets', 'Total Current Liabilities', 'Total Non-Current Assets', 'Total Reserves and Surplus', 'Total Share Capital', 'Total Shareholders Funds', 'Fixed Assets', 'Other Current Liabilities', 'Tangible Assets']
12


In [262]:
not_common_parameters = []

for item in df_parameters:
    if item not in Cash_Flow:
        not_common_parameters.append(item)
        
print(not_common_parameters)
print(len(not_common_parameters))

['nan', 'Adjustments On Amalgamation Merger Demerger Others']
2


In [263]:
index_to_drop =df[df.iloc[:, 0].isin(not_common_parameters)].index

In [264]:
df = df.drop(index_to_drop,axis=0)

In [265]:
df.head(5)

,Cash Flow of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,...,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths
1,Net Profit/Loss Before Extraordinary Items And...,161.40,104.14,51.05,41.00,19.87,-417.92,-379.45,-165.25,108.15,...,250.85,-86.71,91.09,-448.00,-343.56,-818.63,52.25,8.43,324.56,127.96
2,Net CashFlow From Operating Activities,356.00,199.67,111.26,5.16,30.62,-72.54,36.45,-81.36,233.73,...,86.52,82.20,113.86,35.39,73.99,14.63,108.00,-57.44,388.69,103.84
3,Net Cash Used In Investing Activities,-425.97,-764.80,-418.79,-74.46,-58.86,2.01,-39.79,-15.79,-152.96,...,2.64,1.29,16.61,2.76,-3.19,-10.34,-105.15,10.09,561.60,16.68
4,Net Cash Used From Financing Activities,73.34,537.35,121.78,307.59,26.50,77.52,-15.03,42.79,-21.28,...,-41.91,-146.08,-56.31,-25.33,-79.32,-16.05,-48.48,-504.03,-380.01,-132.75


In [266]:
df.replace("12 mths",np.nan,inplace=True)
df.head(3)

,Cash Flow of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Net Profit/Loss Before Extraordinary Items And...,161.40,104.14,51.05,41.00,19.87,-417.92,-379.45,-165.25,108.15,...,250.85,-86.71,91.09,-448.00,-343.56,-818.63,52.25,8.43,324.56,127.96
2,Net CashFlow From Operating Activities,356.00,199.67,111.26,5.16,30.62,-72.54,36.45,-81.36,233.73,...,86.52,82.20,113.86,35.39,73.99,14.63,108.00,-57.44,388.69,103.84


In [267]:
df.dropna(inplace=True)

In [268]:
df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True)


In [269]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')


In [270]:
column_start = df.columns[0]
column_start

'Cash Flow of 3i Infotech(in Rs. Cr.)'

In [271]:
df.columns[:]


Index(['Cash Flow of 3i Infotech(in Rs. Cr.)', 'Mar 09', 'Mar 08', 'Mar 07',
       'Mar 06', 'Mar 05', 'Mar 14', 'Mar 13', 'Mar 12', 'Mar 11', 'Mar 10',
       'Mar 19', 'Mar 18', 'Mar 17', 'Mar 16', 'Mar 15', 'Mar 24', 'Mar 23',
       'Mar 22', 'Mar 21', 'Mar 20'],
      dtype='object')

In [272]:
# Extract the numeric part (years) and sort
sorted_columns = sorted(df.columns[1:], key=lambda x: int(x.split()[1]))

# Reorder the DataFrame columns
df = df[[column_start] + sorted_columns]

# Display the first few rows
df.head(3)


,Cash Flow of 3i Infotech(in Rs. Cr.),Mar 05,Mar 06,Mar 07,Mar 08,Mar 09,Mar 10,Mar 11,Mar 12,Mar 13,...,Mar 15,Mar 16,Mar 17,Mar 18,Mar 19,Mar 20,Mar 21,Mar 22,Mar 23,Mar 24
1,Net Profit/Loss Before Extraordinary Items And...,19.87,41.0,51.05,104.14,161.4,136.49,108.15,-165.25,-379.45,...,-343.56,-448.0,91.09,-86.71,250.85,127.96,324.56,8.43,52.25,-818.63
2,Net CashFlow From Operating Activities,30.62,5.16,111.26,199.67,356.0,194.8,233.73,-81.36,36.45,...,73.99,35.39,113.86,82.2,86.52,103.84,388.69,-57.44,108.0,14.63
3,Net Cash Used In Investing Activities,-58.86,-74.46,-418.79,-764.8,-425.97,-386.0,-152.96,-15.79,-39.79,...,-3.19,2.76,16.61,1.29,2.64,16.68,561.6,10.09,-105.15,-10.34


In [273]:
df = df.T
df.head(3)

,1,2,3,4,5,7,8,9
Cash Flow of 3i Infotech(in Rs. Cr.),Net Profit/Loss Before Extraordinary Items And...,Net CashFlow From Operating Activities,Net Cash Used In Investing Activities,Net Cash Used From Financing Activities,Foreign Exchange Gains / Losses,Net Inc/Dec In Cash And Cash Equivalents,Cash And Cash Equivalents Begin of Year,Cash And Cash Equivalents End Of Year
Mar 05,19.87,30.62,-58.86,26.5,0.0,-1.74,9.66,7.92
Mar 06,41.0,5.16,-74.46,307.59,0.0,238.29,7.92,246.21


In [ ]:
# df = df.T.reset_index()
# df.head(3)+

In [275]:
df.to_excel(output_file,index=False)